In [ ]:
import pandas as pd
import json
import streamlit as st
import wget
import sqlite3

### captura dos dados

In [ ]:
countries = pd.read_json('https://gco.iarc.fr/causes/obesity/data/countries.json')
cancers = pd.read_json('https://gco.iarc.fr/causes/obesity/data/cancers.json')

In [ ]:
attributable = wget.download('https://gco.iarc.fr/causes/obesity/data/attributable.json', out='../data/attributable.json')
preventable = wget.download('https://gco.iarc.fr/causes/obesity/data/preventable.json', out='../data/preventable.json')

In [ ]:
with open('../data/attributable.json') as json_data:
    data = json.load(json_data)
    attributable = pd.json_normalize(data['both'])

In [ ]:
with open('../data/preventable.json') as json_data:
    data = json.load(json_data)
    preventable = pd.json_normalize(data['both'])

In [ ]:
attributable = attributable[['total_m', 'total_f', 'country.name', 'country.WHO_NAME', 'cancer.name']]
attributable = attributable.rename(columns={"country.name":"country", "country.WHO_NAME":"continent", "cancer.name":"cancer"})
attributable['continent'] = attributable['continent'].apply(lambda x: x.replace("WHO ",""))

In [ ]:
preventable = preventable[['total_m', 'total_f', 'country.name', 'country.WHO_NAME', 'cancer.name']]
preventable = preventable.rename(columns={"country.name":"country", "country.WHO_NAME":"continent", "cancer.name":"cancer"})
preventable['continent'] = preventable['continent'].apply(lambda x: x.replace("WHO ",""))

In [ ]:
preventable.head()

In [ ]:
attributable.head()

In [ ]:
con = sqlite3.connect("../data/tutorial.db")

In [ ]:
attributable.to_sql('attributable', con=con, if_exists='replace')
preventable.to_sql('preventable', con=con, if_exists='replace')